In [4]:
import os
import math
import re
import string

no_spam = 0
no_ham = 0
vocabulary = set()
spam_words_dict = {}
ham_words_dict = {}

# read training files
def get_data(file_location, target):
    global no_spam, no_ham
    
    files = os.listdir(file_location)
    for file in files:
        f = open(os.path.join(file_location, file), 'r', encoding = "ISO-8859-1")
        content = f.read()

        #only count the meaningful words
        content = re.sub('[^a-zA-Z]', ' ', content)
        content = content.strip().lower()
        
        for word in content.split():
            vocabulary.add(word)
            
            if target:
                no_spam += 1
                count = spam_words_dict.get(word)
                if count is None:
                    count = 0
                count += 1
                spam_words_dict.update({word: count}) 
                
            else :
                no_ham += 1
                count = ham_words_dict.get(word)
                if count is None:
                    count = 0
                count += 1
                ham_words_dict.update({word: count})


# test the NB with test files
def predict_model(file_location, target):
    correct_ans = 0
    total = 0
    
    files = os.listdir(file_location)
    for file in files:
        total += 1
        prob_spam = math.log2(1/2)
        prob_ham = math.log2(1/2)
        
        f = open(os.path.join(file_location, file), 'r', encoding = "ISO-8859-1")
        content = f.read()
        
        #only count the meaningful words
        content = re.sub('[^a-zA-Z]', ' ', content)
        content = content.strip().lower()
        
        for word in content.split():            
            if spam_words_dict.get(word) is None:
                prob_spam += math.log2(1 / (no_spam + len(vocabulary)))
            else:
                prob_spam += math.log2((spam_words_dict.get(word) + 1) / (no_spam + len(vocabulary)))
                
            if ham_words_dict.get(word) is None:
                prob_ham += math.log2(1 / (no_ham + len(vocabulary)))
            else:
                prob_ham += math.log2((ham_words_dict.get(word) + 1) / (no_ham + len(vocabulary)))
        
        if target:
            if prob_spam > prob_ham:
                correct_ans += 1
        else:
            if prob_ham > prob_spam:
                correct_ans += 1

    return correct_ans, total
    

#main function
if __name__ == '__main__':
    train_spam_location = 'assignment3_train/train/spam'
    train_ham_location = 'assignment3_train/train/ham'
    
    get_data(train_spam_location, True)
    get_data(train_ham_location, False)
    
    test_spam_location = 'assignment3_test/test/spam'
    test_ham_location = 'assignment3_test/test/ham'
    
    spam_correct, spam_total = predict_model(test_spam_location, True)
    ham_correct, ham_total = predict_model(test_ham_location, False)
    
    accuracy = (spam_correct + ham_correct) / (spam_total + ham_total) 
    print ("Naive Bayes Accuracy with stopwords:", accuracy)

Naive Bayes Accuracy with stopwords: 0.9665271966527197
